# **模型訓練（迴歸問題）**
此份程式碼會講解針對迴歸型任務在模型訓練上需要注意的細節。

## 本章節內容大綱
* ### [創建資料集／載入資料集（Dataset Creating/ Loading）](#DatasetCreating/Loading)
* ### [資料前處理（Data Preprocessing）](#DataPreprocessing)
* ### [模型建置（Model Building）](#ModelBuilding)
* ### [模型訓練（Model Training）](#ModelTraining)
* ### [模型評估（Model Evaluation）](#ModelEvaluation)
---

## 匯入套件

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Tensorflow 相關套件
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

<a name="DatasetCreating/Loading"></a>
## 創建資料集／載入資料集（Dataset Creating / Loading）

In [ ]:
# 上傳資料
!wget -q https://github.com/TA-aiacademy/course_3.0/releases/download/DL/Data_part2.zip
!unzip -q Data_part2.zip

In [ ]:
train_df = pd.read_csv('./Data/FilmRating_train.csv')
test_df = pd.read_csv('./Data/FilmRating_test.csv')

In [ ]:
train_df.head()

* #### 電影評價資料集
資料集總共 2612 筆，
欄位包括預算 (budget)、電影類型 (genres)、關鍵字詞 (keywords)、知名度 (popularity)、製作公司 (production_companies)、國家 (production_countries)、收入 (revenue)、時長 (runtime)、卡司 (cast)、導演 (director)、距離發布時間 (n_days)、評分 (score)，多項欄位是以 leave-one-out encoding 方式轉換數值。


In [ ]:
X_df = train_df.iloc[:, :-1].values
y_df = train_df.score.values

In [ ]:
X_test = test_df.iloc[:, :-1].values
y_test = test_df.score.values

<a name="DataPreprocessing"></a>
## 資料前處理（Data Preprocessing）

* ### 資料正規化（Data Normalization）
    - 減少過度關注的特徵（由特定數字範圍造成的影響）
    - 避免更新方向偏離，較容易收斂

對於測試資料，需使用「訓練資料」的統計量去做轉換，避免改變兩組資料間的分布關係
![](https://i.imgur.com/BqMLf3P.png)

In [ ]:
'''Normalize'''
X_scale = (X_df-X_df.min(axis=0)) / (X_df.max(axis=0)-X_df.min(axis=0))
X_test_scale = (X_test-X_df.min(axis=0)) / (X_df.max(axis=0)-X_df.min(axis=0))

# 其他寫法
# from sklearn.preprocessing import MinMaxScaler
# sc = MinMaxScaler(feature_range=(0, 1))
# X_scale = sc.fit_transform(X_df)
# X_test_scale = sc.transform(X_test)

# '''Standardize'''
# X_scale = (X_df-X_df.mean(axis=0)) / (X_df.std(axis=0))
# X_test_scale = (X_test-X_df.mean(axis=0)) / (X_df.std(axis=0))

# 其他寫法
# from sklearn.preprocessing import StandardScaler
# sc = StandardScaler()
# X_scale = sc.fit_transform(X_df)
# X_test_scale = sc.transform(X_test)

* ### 資料切分（Data Splitting）

In [ ]:
# train, valid/test dataset split
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = \
    train_test_split(X_scale, y_df, test_size=0.1, random_state=17)

In [ ]:
print(f'X_train shape: {X_train.shape}')
print(f'X_valid shape: {X_valid.shape}')
print(f'y_train shape: {y_train.shape}')
print(f'y_valid shape: {y_valid.shape}')

<a name="ModelBuilding"></a>
## 模型建置（Model Building）

In [ ]:
keras.backend.clear_session()  # 重置 keras 的所有狀態
tf.random.set_seed(17)  # 設定 tensorflow 隨機種子

model = keras.models.Sequential()
model.add(layers.Dense(64,  # 神經元個數
                       input_shape=X_train[0].shape,  # 輸入形狀
                       activation='sigmoid'))  # 激活函數
model.add(layers.Dense(32, activation='sigmoid'))
model.add(layers.Dense(1, activation='linear'))

model.summary()

![](https://i.imgur.com/mvz49nJ.png)

<a name="ModelTraining"></a>
## 模型訓練（Model Training）

* ### 模型編譯（model compile）
設定模型訓練時，所需的優化器 (optimizer)、損失函數 (loss function)

In [ ]:
model.compile(optimizer='rmsprop',  # default: RMSprop(learning_rate=0.001)
              loss='mean_squared_error')

![](https://i.imgur.com/L07x8qA.png)

In [ ]:
history = model.fit(X_train, y_train,
                    epochs=20,
                    batch_size=8,
                    validation_data=(X_valid, y_valid))

<a name="ModelEvaluation"></a>
## 模型評估（Model Evaluation）

* ### 視覺化訓練過程的評估指標 （Visualization）

In [ ]:
# type(history.history) = dictionary
print(history.history.keys())

In [ ]:
train_loss = history.history['loss']
valid_loss = history.history['val_loss']

In [ ]:
plt.figure(figsize=(15, 4))
plt.yscale('log')
plt.plot(range(len(train_loss)), train_loss, label='train_loss')
plt.plot(range(len(valid_loss)), valid_loss, label='valid_loss')

plt.legend()
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.show()

* ### 模型預測（Model predictions）

In [ ]:
y_pred = model(X_valid)
print(f'預測結果： {y_pred[:5, 0]}')
print(f'目標值： {y_valid[:5]}')

* ### 視覺化結果

In [ ]:
plt.figure(figsize=(15, 4))
plt.plot(range(len(y_pred)), y_pred, label='prediction')
plt.plot(range(len(y_valid)), y_valid, label='groundtruth')
plt.plot(range(len(y_pred)), y_pred[:, 0]-y_valid, label='difference')

plt.legend()
plt.xlabel('Samples')
plt.ylabel('Values')
plt.show()